# SanFranciso Crime Classification

Dataset: [kaggle](https://www.kaggle.com/c/sf-crime/data)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')





In [ ]:
#날짜를 칼럼별로 나눔
train['Date'] = pd.to_datetime(train['Dates'].dt.date)
train['n_days'] = (train['Date'] - train['Date'].min()).apply(lambda x: x.days)
train['Day'] = train['Dates'].dt.day
train['DayOfWeek'] = train['Dates'].dt.weekday
train['Month'] = train['Dates'].dt.month
train['Year'] = train['Dates'].dt.year
train['Hour'] = train['Dates'].dt.hour
train['Minute'] = train['Dates'].dt.minute
train['Block'] = train['Address'].str.contains('block', case=False).apply(lambda x: 1 if x == True else 0)
train['ST'] = train['Address'].str.contains('ST', case=False).apply(lambda x: 1 if x == True else 0)
train["X_Y"] = train["X"] - train["Y"]
train["XY"] = train["X"] + train["Y"]

In [ ]:
test['Date'] = pd.to_datetime(test['Dates'].dt.date)
test['n_days'] = (test['Date'] - test['Date'].min()).apply(lambda x: x.days)
test['Day'] = test['Dates'].dt.day
test['DayOfWeek'] = test['Dates'].dt.weekday
test['Month'] = test['Dates'].dt.month
test['Year'] = test['Dates'].dt.year
test['Hour'] = test['Dates'].dt.hour
test['Minute'] = test['Dates'].dt.minute
test['Block'] = test['Address'].str.contains('block', case=False).apply(lambda x: 1 if x == True else 0)
test['ST'] = test['Address'].str.contains('ST', case=False).apply(lambda x: 1 if x == True else 0)
test["X_Y"] = test["X"] - test["Y"]
test["XY"] = test["X"] + test["Y"]


In [ ]:
# 문자를 숫자로 바꾼다. 머신러닝 모델에 넣을수 있기위해
#1 라이브러리 import 
from sklearn.preprocessing import LabelEncoder

#2 객체를 불러온다
le1 = LabelEncoder()

#3 lookuptable 보고 함수 실행
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])

le2 = LabelEncoder()
y= le2.fit_transform(train['Category'])#종속 변수 분리 + 문자->숫자 변경

#address 는 둘다 train, test 에 lookup table을 만든다.
le3 = LabelEncoder()
le3.fit(list(train['Address']) + list(test['Address']))
# lookuptable 참조하여 transform
train['Address'] = le3.transform(train['Address'])
test['Address'] = le3.transform(test['Address'])



In [ ]:
#쓸데없는 칼럼 드랍

train.drop(['Dates','Date','Descript','Resolution', 'Category'], 1, inplace=True)
test.drop(['Dates','Date',], 1, inplace=True)

In [ ]:
# Classifier 모델 선정 기준 1. 데이터셋에 굉장히 많은 문자열을 포함하고있다. 2. 데이터가 어느정도 대용량이다.
# LGBM 모델 선정 이유 1. 문자열 데이터를 잘 처리함 2. 데이터가 대용량 사용 가능(데이터가 크면 클수록 속도가 더 빨라지는 특징) 3. tree모델은 데이터가 분류가 되기만 한다면 학습을 잘함. classifier에 특화
# 문자열 데이터를 잘 처리하는 모델: 1)Catboost(문자열 가장 잘 처리, 데이터 적은량에 쓴다 ,데이터가 많으면 시간이 많이 걸린다)
# 문자열 데이터 다룰때 => 1. label encoding 을 해야겠다. 2. tree 모델을 서야겠다.(tree모델은 데이터가 분류가 되기만 한다면)

#1. 모델을 불러온다: 분류 할때 classifier 쓴다
from lightgbm import LGBMClassifier
#2. 모델을 선언한다.
model = LGBMClassifier(objective="multiclass", num_class=39, max_bin = 465, max_delta_step = 0.9,
                      learning_rate=0.4, num_leaves = 42, n_estimators=100,)
#3. 모델을 학습시킨다. 누가 카데고리 feature인지 알려줌,/ address를 카데고리 feature에 안넣어진이유: 너무 많은 unique 갯수를 가지고 있음 => 학습시간이 길다., 성
model.fit(train, y, categorical_feature=["PdDistrict", "DayOfWeek"])
#4. 모델을 예측한다. / classifer는 proba(확률)를 붙힘 - Log loss로 평가하기때문에 label로 예측해버리면 틀렸을때 패널티가 너무 크기때문에 Proba(확률)로 예측함
preds = model.predict_proba(test)

In [ ]:
#le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')  => 역으로 숫자에서 문자로 바꿈
submission = pd.DataFrame(preds, columns=le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')), index=test.index)
submission.to_csv('LGBM_final.csv', index_label='Id')